In [ ]:
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from joblib import Parallel, delayed
import meshio  # Replace PyVista with Meshio

# Load Volume Mesh using Meshio
def load_volume_mesh(filepath):
    print("Loading volume mesh...")
    mesh = meshio.read(filepath)
    print("Volume mesh loaded successfully.")
    
    # Extract points and cells from the mesh
    points = np.array(mesh.points)
    cells = np.array(mesh.cells_dict['tetra'])  # Assuming a tetrahedral mesh
    return points, cells

# Assign Material Properties
def assign_material_properties(cells, cohesion, friction_angle, density, E, nu):
    print("Assigning material properties...")
    
    # Create arrays to store material properties for each cell
    material_props = {
        'cohesion': np.full(cells.shape[0], cohesion),
        'friction_angle': np.full(cells.shape[0], friction_angle),
        'density': np.full(cells.shape[0], density),
        'E': np.full(cells.shape[0], E),
        'nu': np.full(cells.shape[0], nu)
    }
    
    print("Material properties assigned.")
    return material_props

# Compute Jacobian
def compute_jacobian(element_nodes, dN_dxi):
    J = np.zeros((3, 3))  # 3x3 for a 3D element
    for i in range(4):  # Assuming a 4-node tetrahedral element
        J += np.outer(dN_dxi[i], element_nodes[i])
    return J

# Compute B Matrix
def compute_B_matrix(J_inv, dN_dxi):
    B = np.zeros((6, 12))  # 6 strains and 3 displacements per node, 4 nodes * 3 = 12
    for i in range(4):
        dN_dx = np.dot(J_inv, dN_dxi[i])
        B[0, i*3] = dN_dx[0]  # ε_xx
        B[1, i*3+1] = dN_dx[1]  # ε_yy
        B[2, i*3+2] = dN_dx[2]  # ε_zz
        B[3, i*3] = dN_dx[1]  # ε_xy
        B[3, i*3+1] = dN_dx[0]
        B[4, i*3+1] = dN_dx[2]  # ε_yz
        B[4, i*3+2] = dN_dx[1]
        B[5, i*3] = dN_dx[2]  # ε_zx
        B[5, i*3+2] = dN_dx[0]
    return B

# Compute C Matrix
def compute_C_matrix(E, nu):
    C = np.zeros((6, 6))  # 6x6 matrix for 3D stress-strain relationship
    factor = E / (1 + nu) / (1 - 2 * nu)
    C[0, 0] = C[1, 1] = C[2, 2] = factor * (1 - nu)
    C[3, 3] = C[4, 4] = C[5, 5] = E / 2 / (1 + nu)
    C[0, 1] = C[1, 0] = C[0, 2] = C[2, 0] = C[1, 2] = C[2, 1] = factor * nu
    return C

# Compute Element Stiffness
def compute_element_stiffness(E, nu, element_nodes):
    dN_dxi = np.array([
        [-1, -1, -1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    J = compute_jacobian(element_nodes, dN_dxi)
    det_J = np.linalg.det(J)
    
    if np.abs(det_J) < 1e-12:
        print("Warning: Degenerate element detected. Skipping element.")
        return None
    
    J_inv = np.linalg.inv(J)
    B = compute_B_matrix(J_inv, dN_dxi)
    C = compute_C_matrix(E, nu)
    K_elem = np.dot(B.T, np.dot(C, B)) * det_J
    return K_elem

# Assemble Global Stiffness Matrix Efficiently
def assemble_global_stiffness_efficient(K_global, K_elem, element):
    num_dofs_per_node = 3
    num_nodes = len(element)
    global_indices = np.array([int(node * num_dofs_per_node) for node in element], dtype=np.int32)
    
    data = []
    rows = []
    cols = []
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            global_i = global_indices[i]
            global_j = global_indices[j]
            
            for k in range(num_dofs_per_node):
                for l in range(num_dofs_per_node):
                    value = K_elem[i*num_dofs_per_node+k, j*num_dofs_per_node+l]
                    data.append(value)
                    rows.append(global_i+k)
                    cols.append(global_j+l)
    
    data = np.array(data)
    rows = np.array(rows)
    cols = np.array(cols)
    
    # Use SciPy's sparse matrix addition
    K_global += sp.coo_matrix((data, (rows, cols)), shape=K_global.shape).tocsr()
    return K_global

# Parallel function to compute stiffness for each cell
def compute_and_assemble_for_cell(E, nu, cell_point_ids, cell_nodes):
    K_elem = compute_element_stiffness(E, nu, cell_nodes)
    if K_elem is not None:
        return (cell_point_ids, K_elem)
    return None

# Compute Global Stiffness Matrix using joblib
def compute_global_stiffness_matrix(points, cells, E, nu):
    print("Computing global stiffness matrix...")

    # Preprocess the cells: extract point_ids and node coordinates
    cells_data = [(cell, points[cell]) for cell in cells]

    # Initialize global stiffness matrix as a sparse COO matrix
    K_global = sp.coo_matrix((points.shape[0] * 3, points.shape[0] * 3))
    
    # Parallel processing over cells
    results = Parallel(n_jobs=-1)(delayed(compute_and_assemble_for_cell)(E, nu, cell_point_ids, cell_nodes) for cell_point_ids, cell_nodes in tqdm(cells_data))
    
    # Combine results
    for result in results:
        if result is not None:
            cell_point_ids, K_elem = result
            K_global = assemble_global_stiffness_efficient(K_global, K_elem, cell_point_ids)

    print("Global stiffness matrix computed.")
    return K_global.tocsr()  # Convert to CSR format after assembly

# --- Main script starts here ---

# Material properties
cohesion = 0  # Cohesion in Pascals
friction_angle = 10  # Friction angle in degrees
density = 1500  # Density in kg/m³
E = 1000  # Young's modulus in Pascals
nu = 0.3  # Poisson's ratio

# Load the volume mesh
filepath = r"C:\Users\shubh\Documents\Analytics\FOS WS\Mesh Files\mesh.vtk"
points, cells = load_volume_mesh(filepath)

# Assign material properties
material_props = assign_material_properties(cells, cohesion, friction_angle, density, E, nu)

# Compute the global stiffness matrix
K_global = compute_global_stiffness_matrix(points, cells, E, nu)

Loading volume mesh...
Volume mesh loaded successfully.
Assigning material properties...
Material properties assigned.
Computing global stiffness matrix...


100%|██████████| 6924336/6924336 [10:48<00:00, 10679.50it/s]
